In [1]:
# Import necessary libraries
from openprompt.plms import T5TokenizerWrapper
from datasets import load_from_disk
from openprompt.pipeline_base import PromptDataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from openprompt.prompts import ManualTemplate, MixedTemplate
from openprompt import PromptForClassification
from openprompt.data_utils import FewShotSampler
from random import shuffle
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup
import torch
from openprompt.prompts import ManualVerbalizer
from openprompt.data_utils import InputExample
from tqdm import tqdm
import json
dataset_path = "/lustre/work/client/users/minhos/cache/datasets/p3_cos_qa"
raw_dataset = load_from_disk(dataset_path)



t5_path = "/lustre/work/client/users/minhos/models_for_supercomputer/t5-base"
model = T5ForConditionalGeneration.from_pretrained(t5_path)
tokenizer = T5Tokenizer.from_pretrained(t5_path)


# Logging setup
log_file = "qa_multiple_choice_id_t5.json"
results = []

label_map = {"A":0, "B":1, "C":2, "D":3 , "E":4}

dataset = {}
for split in ['train', 'validation']:
    dataset[split] = []
    if split == 'train':
        raw_dataset[split] = raw_dataset[split].shuffle(seed=42).select(range(1000))
    else:
        raw_dataset[split] = raw_dataset[split].select(range(500))
    
    for idx, data in enumerate(raw_dataset[split]):
        label_text = data["targets_pretokenized"].strip()
        label_numeric = label_map.get(label_text, -1)
        input_example = InputExample(text_a=data['inputs_pretokenized'], guid=idx, label=label_numeric)
        dataset[split].append(input_example)
print(dataset['train'][0])
print(type(dataset['train'][0]))


sampler = FewShotSampler(num_examples_per_label=30)
fewshot_data = sampler(dataset['train'], seed=42)
def evaluate(prompt_model, dataloader):
    prompt_model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    
    with torch.no_grad():
        for inputs in dataloader:
            logits = prompt_model(inputs)
            preds = torch.argmax(logits, dim=-1)
            labels = inputs['label']
            
            total += len(labels)
            correct += (preds == labels).sum().item()
        
    accuracy = correct / total
    return accuracy

# Hyperparameter search ranges
learning_rates = [0.005, 0.001, 0.0005] # 0.0005, 0.001, 0.005
warmup_steps = [10]

for lr in learning_rates:
    for warmup in warmup_steps:

        model = T5ForConditionalGeneration.from_pretrained(t5_path)
        tokenizer = T5Tokenizer.from_pretrained(t5_path)

        template = ManualTemplate(
            tokenizer=tokenizer,
            text='{"placeholder":"text_a"} Which option is correct? {"mask"}',
        )
        verbalizer = ManualVerbalizer(
            tokenizer=tokenizer,
            num_classes=5,
            label_words=[
                ["A", "a", "Option A", "first choice"],
                ["B", "b", "Option B", "second choice"],
                ["C", "c", "Option C", "third choice"],
                ["D", "d", "Option D", "fourth choice"],
                ["E", "e", "Option E", "fifth choice"]
            ]
        )
        wrapped_example = template.wrap_one_example(dataset['train'][0])
        prompt_model = PromptForClassification(
            plm=model,
            template=template,
            verbalizer=verbalizer,
            freeze_plm=False,
        )
        train_dataloader = PromptDataLoader(
            dataset = fewshot_data,
            template=template,
            tokenizer=tokenizer,
            tokenizer_wrapper_class=T5TokenizerWrapper,
            decoder_max_length=3, max_seq_length=480,
            batch_size=5)


        validation_dataloader = PromptDataLoader(
            dataset = dataset["validation"],
            template=template,
            tokenizer=tokenizer,
            tokenizer_wrapper_class=T5TokenizerWrapper,
            decoder_max_length=3, max_seq_length=480,
            batch_size=20
        )

        loss_func = torch.nn.CrossEntropyLoss()
        no_decay = ['bias', 'LayerNorm.weight']
        # it's always good practice to set no decay to biase and LayerNorm parameters
        optimizer_grouped_parameters = [
            {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

        from tqdm import tqdm
        optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
        prompt_model.train()
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup, num_training_steps=1000)
        for epoch in range(10):
            total_loss = 0
            pbar = tqdm(train_dataloader, desc="Training")
            for step, inputs in enumerate(train_dataloader):
                logits = prompt_model(inputs)
                labels = inputs['label']
                loss = loss_func(logits, labels)
                loss.backward()
                total_loss += loss.item()
                optimizer.step()
                optimizer.zero_grad()
                pbar.set_postfix({"loss": total_loss / (step + 1)})
                if step %100 ==1:
                    print("Epoch {}, average loss: {}".format(epoch+1, total_loss/(step+1)), flush=True)

       
    
            # Validation after each epoch
        val_accuracy = evaluate(prompt_model, validation_dataloader)
        print(f"Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.4f}")
        # Log results
        result = {
            "learning_rate": lr,
            "warmup_steps": warmup,
            "final_loss": total_loss / (10 * len(train_dataloader)),
            "accuracy": val_accuracy
        }
        results.append(result)
        
        # Save results to JSON
        with open(log_file, "w") as f:
            json.dump(results, f, indent=4)
            
print("Tuning complete. Results saved to", log_file)

/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbit

{
  "guid": 0,
  "label": 3,
  "meta": {},
  "text_a": "Pick the option in line with common sense to answer the question.\nQuestion: It was the only way out of town, the police parked their vehicles and drew their guns to create a what?\nOptions:\n\nA. war\n\nB. sporting goods store\n\nC. military base\n\nD. roadblock\n\nE. fun\n\n",
  "text_b": "",
  "tgt_text": null
}

<class 'openprompt.data_utils.utils.InputExample'>


tokenizing: 150it [00:00, 1147.18it/s]
tokenizing: 500it [00:00, 1422.81it/s]
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training:   0%|          | 0/30 [00:13<?, ?it/s, loss=1.59]

Epoch 1, average loss: 1.5899412631988525


Training:   0%|          | 0/30 [01:24<?, ?it/s, loss=1.67]

Training:   0%|          | 0/30 [00:05<?, ?it/s, loss=1.7] 

...

Epoch 9, average loss: 1.5478777885437012


Training:   0%|          | 0/30 [07:25<?, ?it/s, loss=1.65]

Training:   0%|          | 0/30 [00:26<?, ?it/s, loss=1.52]

Epoch 10, average loss: 1.5236400961875916



Training:   0%|          | 0/30 [04:48<?, ?it/s, loss=1.67]

Validation Accuracy after Epoch 10: 0.1960
Tuning complete. Results saved to qa_multiple_choice_id_t5.json
